<h3>Inspiration: &lt;&lt;Hidden Order: How Adaptation Builds Complexity&gt;&gt; by Holland (ISBN: 9780201407938) outlined the basics of a genetic algorithm to simulate the Prisoner's dilemma.</h3>

<h3><center>Use genetic programming to yield the most desirable strategy for prisoner's dilemma</center></h3>
<br>
<center><h4>A Standard Pay-off Matrix</h4></center>

<table>
    <tr>
        <td></td>
        <td></td>
        <td colspan=2>Prisoner B</td>
    </tr>
    <tr>
        <td></td>
        <td></td>
        <td>Cooperate</td>
        <td>Defect</td>
    </tr>
    <tr>
        <td rowspan=2>Prisoner A</td>
        <td>Cooperate</td>
        <td>(3, 3)</td>
        <td>(-5, 10)</td>
    </tr>
    <tr>
        <td>Defect</td>
        <td>(10, -5)</td>
        <td>(-1, -1)</td>
    </tr>
</table>


<h3>Based on some evolutionary literatures, I decide to start w/ an example genetic algorithm from scratch.</h3>

<h3>Some excellent reads:</h3>
<ul>
    <li><b>&lt;&lt;On the Origin of Species&gt;&gt; by Charles Darwin. Of course this has to be the 1st book on the list. A remarkably careful thinker w/ extremely insightful and well-articulated ideas.</b></li>
    <li>&lt;&lt;The Red Queen: Sex and the Evolution of Human Nature&gt;&gt; by M. Ridley</li>
    <li>&lt;&lt;The Third Chimpanzee: The Evolution and Future of the Human Animal&gt;&gt; by J. Diamond</li>
    <li>&lt;&lt;Hidden Order: How Adaptation Builds Complexity&gt;&gt; by J. Holland</li>
    <li>&lt;&lt;Why We Get Sick: The New Science of Darwinian Medicine&gt;&gt; by R. Nesse</li>
    <li>&lt;&lt;Psychology: An Evolutionary Approach&gt;&gt; by S. Gaulin and D. McBurney</li>
    <li>&lt;&lt;The Ancestor's Tale&gt;&gt; by R. Dawkin</li>
</ul>

<h3>Theoretically, given enough sample size and computing power, a forgiving tic-for-tac strategy should win out. I'm more interested in playing w/ multiple different payoff matrixes and evolutionary scenarioes.</h3>

<h3>Note: any sensible scientist will bark at me for this, but "strategy" and "individual" will be used interchangeably in this example, since I assume each individual is only capable of one fixed pattern of strategy. It's simply a starting point of building a model.</h3>

In [1]:
import random
import numpy as np
from collections import Counter
import warnings
import pandas as pd
from tqdm import tqdm_notebook
from IPython.display import display, HTML
pd.set_option('max_colwidth', 100)

In [2]:
# Pay-off matrix
matrix = {'CC': (3, 3),
          'CD': (-5, 10),
          'DC': (10, -5),
          'DD': (-1, -1)}

def display_matrix(matrix):
    df = pd.DataFrame({'Action': ['Cooperate', 'Defect'],
                       'Cooperate': [matrix['CC'], matrix['DC']],
                       'Defect': [matrix['CD'], matrix['DD']]})
    display(HTML(df.to_html()))
    
display_matrix(matrix)

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-5, 10)"
1,Defect,"(10, -5)","(-1, -1)"


In [3]:
# To start w/ simple parameters

past_n = 5 # past n interaction to remember for agent
total_n = 5 # total # of interactions remembered by agent
evolution_cycles = 10 # evolution cycles #
population_size = 5 # population size
max_competitions_per_round = 5 # Number of competitions per round
mutation_rate = 0.01 # probability for any one of the strategies to mutate
strats = ['C', 'D'] # Cooperate or Defect

### We need a way to calculate rewards in each round, based on the pay-off matrix

In [4]:
def calc_rewards(strat_list_1,
                 strat_list_2,
                 past_n = past_n,
                 matrix = matrix,
                 agent_1_only = True):
    if len(strat_list_1) != len(strat_list_2):
        warnings.warn("strat_lists must be of equal length!")
        return
    else:

        total_score_1, total_score_2 = 0, 0
        strat_list_zip = zip(strat_list_1[-past_n:], strat_list_2[-past_n:])

        if agent_1_only:
            for move_1, move_2 in strat_list_zip:
                score_1 = matrix[move_1+move_2][0]

                total_score_1 += score_1

            return total_score_1
        
        else:
            for move_1, move_2 in strat_list_zip:
                score_1 = matrix[move_1+move_2][0]
                score_2 = matrix[move_1+move_2][1]

                total_score_1 += score_1
                total_score_2 += score_2

            return total_score_1, total_score_2

In [5]:
random_strat_list_1 = random.choices(strats,
                                     k=3)
random_strat_list_2 = random.choices(strats,
                                     k=3)
print(random_strat_list_1)
print(random_strat_list_2)

calc_rewards(strat_list_1=random_strat_list_1,
             strat_list_2=random_strat_list_2,
             agent_1_only=False)

['D', 'D', 'C']
['C', 'C', 'C']


(23, -7)

<h2><center>So, we'll start with this pay-off matrix</center></h2>
<table>
    <tr>
        <td></td>
        <td colspan=3>Prisoner B</td>
    </tr>
    <tr>
        <td></td>
        <td></td>
        <td>Cooperate</td>
        <td>Defect</td>
    </tr>
    <tr>
        <td rowspan=2>Prisoner A</td>
        <td>Cooperate</td>
        <td>(3, 3)</td>
        <td>(-5, 10)</td>
    </tr>
    <tr>
        <td>Defect</td>
        <td>(10, -5)</td>
        <td>(-1, -1)</td>
    </tr>
</table>

In [6]:
# A helpful way to condense the strategies employed by each individual:

def stringify_list(input_list,
                   joint=''):
    return joint.join(input_list)

stringify_list(['D', 'C', 'D', 'D', 'D'])

'DCDDD'

In [7]:
# A helpful way to visualize cross overs & mutations during evolution

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

print('Father: CC' + color.BLUE + color.BOLD + 'DDD' + color.END)
print('Mother: ' + color.BLUE + color.BOLD + 'DD' + color.END + 'CCC')
print('Child:  ' + color.BLUE + color.BOLD + 'DDDDD' + color.END)

Father: CCDDD
Mother: DDCCC
Child:  DDDDD


### We need a way to repopulate the next generation based on the relative fitness of the previous generation. We'll keep the overall population size constant at each generation.

In [8]:
def count_c(row,
            rounding = 2):
    return(round(Counter(row['Strategy'])['C'] / len(row['Strategy']), rounding))

def repopulate(population_list,
               total_score_list,
               population_size,
               render_stat_df=True,
               ordered_df=True):
    if len(population_list) != population_size:
        warnings.warn(f'New population has a size of {population_size}, different from the previous population {len(population_list)}!')
    else:
        new_population_list = random.choices(population_list,
                                             weights=total_score_list,
                                             k = population_size)
        
        if not render_stat_df:
            return new_population_list
        
        else:
            # Represent the strat_lists in string format for easier viewing
            population_str_list = [''.join(x) for x in population_list]
            new_population_str_list = [''.join(x) for x in new_population_list]
            
            # Consolidate identical strat_lists
            population_counter = Counter(population_str_list)
            
            stat_df = pd.DataFrame(data={'Strategy': population_str_list,
                                         'Score': total_score_list})
            stat_df_grouped = stat_df.groupby(by='Strategy') \
                                     .agg({'Strategy': 'size',
                                           'Score': 'mean'}) \
                                     .rename(columns={'Strategy': 'Freq',
                                                      'Score': 'Avg_wt'}) \
                                     .reset_index()
            stat_df_grouped['Avg_wt'] = round(stat_df_grouped['Avg_wt'], 2)
            
            new_population_counter = Counter(new_population_str_list)
            
            new_population_df = pd.DataFrame.from_dict(new_population_counter, orient='index') \
                                            .reset_index()
            new_population_df.rename(columns={'index': 'Strategy',
                                              0: 'New_freq'},
                                     inplace=True)
            final_stat_df = stat_df_grouped.merge(new_population_df,
                                                  on = 'Strategy',
                                                  how='left')
            final_stat_df['New_freq'] = final_stat_df['New_freq'].fillna(0)
            final_stat_df['New_freq'] = final_stat_df['New_freq'].astype(int)
            
            final_stat_df['C_%'] = final_stat_df.apply(count_c, axis=1)
            
            final_stat_df.sort_values(by='Avg_wt',
                                      ascending=False,
                                      inplace=True)
            final_stat_df.reset_index(drop=True,
                                      inplace=True)
            
            return new_population_list, final_stat_df
        
            

In [9]:
initial_test_population = [['D', 'D', 'C', 'C', 'C'],
                           ['D', 'D', 'C', 'C', 'C'],
                           ['C', 'D', 'D', 'C', 'C'],
                           ['C', 'D', 'D', 'C', 'C']]

population_snapshot, strat_df = repopulate(population_list = initial_test_population,
                                           total_score_list = [20,23,9,6],
                                           population_size= 4)
print('Population:')
for individual in population_snapshot:
    print(individual)

display(HTML(strat_df.to_html()))


Population:
['D', 'D', 'C', 'C', 'C']
['C', 'D', 'D', 'C', 'C']
['D', 'D', 'C', 'C', 'C']
['D', 'D', 'C', 'C', 'C']


,Strategy,Freq,Avg_wt,New_freq,C_%
0,DDCCC,2,21.5,3,0.6
1,CDDCC,2,7.5,1,0.6


### We need a way to simulate mutation at each generation. It's helpful to specify two customizable parameters:
### 1. mutation_rate: this governs the likelihood of mutation within a new generation
### 2. min_mutation_individual: the minumum number of individuals undergoing mutation at each new generation. This can be helpful in consistently steering population from homogeneity, therefore challenging predominate strategies to novel strategies

In [10]:
def mutate(population_list,
           mutation_rate,
           min_mutation_individual=1,
           verbose=1):
    # for efficiency's sake, generate random # for at an individual level. If lower than threshold, then change one random element from list
    if min_mutation_individual > len(population_list):
        warnings.warn(f'min_mutation_individual {min_mutation_individual} cannot be greater than population: {population_list}!')
        return
    if verbose > 0:
        print(f'\nMutation:')
    mutation_counter = 0
    mutation_index = []
    for index, population in enumerate(population_list):
        if len(population_list) - index <= min_mutation_individual - mutation_counter: # Ensure a min number of mutations
            mutation = True
        else:
            mutation = random.uniform(0, 1) < mutation_rate
        if mutation:
            mutation_counter += 1
            mutation_index.append(index)
            mutation_site = random.randint(0, len(population) -1)
            strats_copy = strats.copy()
            strats_copy.remove(population[mutation_site])
            new_strat = random.choice(strats_copy)
            if verbose > 0:
                print(f'{stringify_list(population[:mutation_site])}' \
                      + color.RED + color.BOLD + f'({population[mutation_site]}->{new_strat})'+ color.END \
                      + f'{stringify_list(population[mutation_site+1:])}')
            population[mutation_site] = new_strat
        
    return population_list
    
mutate(population_list=initial_test_population,
       min_mutation_individual=1,
       mutation_rate=0)


Mutation:
CDD(C->D)C


[['D', 'D', 'C', 'C', 'C'],
 ['D', 'D', 'C', 'C', 'C'],
 ['C', 'D', 'D', 'C', 'C'],
 ['C', 'D', 'D', 'D', 'C']]

### We need to simulate crossover at each generation. Strictly speaking, this is not necessary. I read one theory that higher-order creatures (i.e. mammals) evolved sexual reproduction to combat parasitic agents. In nature, asexual reproduction is also widely seen (i.e. bacteria). Some species alternate between sexual & asexual reproduction means. For furter readings, I recommend &lt;&lt;The Red Queen&gt;&gt; by M. Ridley (ISBN: 9780060556570).

### Since in this simulation, there's no parasitic agents to avoid, crossover may not be necessary. But crossover can introduce hybrids that enhances genetic variation, as well as potentially passing down suitable building blocks of genes, since useful building blocks are theoretically more likely to be present in future generations, so a crossover mechanism essentially allows more building w/ more useful "lego blocks". See &lt;&lt;Hidden Order&gt;&gt; by J. Holland for a better illustration of this idea.

In [11]:
def crossover(population_list,
              crossover_rate,
              verbose=1):
    num_of_crossover = int(crossover_rate * len(population_list))
    if verbose >= 2:
        print(f'\n{num_of_crossover} of {len(population_list)} crossovers.')
    random.shuffle(population_list)
    crossover_population = population_list[0: num_of_crossover]
    final_population = population_list[num_of_crossover:]
    
    # Randomly pair up individual in 1:1 'male-female' ratio. Allow self-pollinating in order to speed-up algorithm (minimal impact when population size is large)
    for individual in crossover_population:
        partner = random.choice(crossover_population)
        crossover_index = random.randint(0, len(individual) - 1)
        new_individual = individual[:crossover_index] + partner[crossover_index:]
        if verbose >= 2:
            print('father: ' + color.BOLD + color.BLUE + stringify_list(individual[:crossover_index]) + color.END + stringify_list(individual[crossover_index:]) + '\n' \
                  'mother: ' + stringify_list(partner[:crossover_index]) + color.BOLD + color.BLUE + stringify_list(partner[crossover_index:]) + color.END + '\n' +\
                  ' child: ' + color.BOLD + color.BLUE + stringify_list(new_individual) + color.END + '\n')
        final_population.append(new_individual)
        
    return final_population

crossover(population_list=initial_test_population,
          crossover_rate=0.9,
          verbose=2)


3 of 4 crossovers.
father: CDDDC
mother: CDDDC
 child: CDDDC

father: CDDCC
mother: CDDCC
 child: CDDCC

father: DDCCC
mother: CDDCC
 child: DDDCC



[['D', 'D', 'C', 'C', 'C'],
 ['C', 'D', 'D', 'D', 'C'],
 ['C', 'D', 'D', 'C', 'C'],
 ['D', 'D', 'D', 'C', 'C']]

### Putting these helper functions together into a genetic algorithm.

In [12]:
past_n = total_n # Allow the agent to remember all past interactions

def genetic_selection(total_n = total_n,
                      past_n = past_n,
                      evolution_cycles = evolution_cycles,
                      population_size = population_size,
                      max_competitions_per_round = max_competitions_per_round,
                      matrix = matrix,
                      mutation_rate = None,
                      min_mutation_individual = 1,
                      crossover_rate = None,
                      verbose = 0,
                      return_counter=True):
    
    population_list = [random.choices(strats,
                                      k = total_n) for _ in range(population_size)]
    
    prog_bar = tqdm_notebook(total=evolution_cycles)
    prog_bar.set_description_str('Evolution Progress:')
    
    display_matrix(matrix)
    # simulate competition & score each strategies
    for generation in range(evolution_cycles):
        total_score_list = []
        if verbose > 0:
            print(f'\n-----\nGeneration: {generation}')
        for population in population_list:
            if verbose >= 3:
                print(f'Individual: {stringify_list(population)}')

            competitor_lists = random.choices(population_list,
                                              k = max_competitions_per_round) # can compete with itself
            total_score_A = 0
            total_score_B = 0
            for competitor in competitor_lists:
                score_A, score_B = calc_rewards(strat_list_1=population,
                                                strat_list_2=competitor,
                                                matrix=matrix,
                                                past_n=past_n,
                                                agent_1_only=False)
                total_score_A += score_A
                total_score_B += score_B
                
                if verbose >=3:
                    print(f'Competitor: {stringify_list(competitor)} | score: {total_score}.')

            total_score_list.append(total_score_A + total_score_B)
        
        if verbose == 1:
            population_str_list = [''.join(x) for x in population_list]
            gen_counter = Counter(population_str_list).most_common()

        # re-populate based on the weighs (total_score_list)
        if verbose <= 0:
            population_list = repopulate(population_list=population_list,
                                         total_score_list=total_score_list,
                                         population_size=population_size,
                                         render_stat_df=False)
        elif verbose > 0:
            population_list, stat_df = repopulate(population_list=population_list,
                                                  total_score_list=total_score_list,
                                                  population_size=population_size,
                                                  render_stat_df=True)

            display(HTML(stat_df.to_html()))

        if crossover_rate is not None:
            population_list = crossover(population_list=population_list,
                                        crossover_rate=crossover_rate,
                                        verbose=verbose)
            
        if mutation_rate is not None:
            population_list = mutate(population_list=population_list,
                                     min_mutation_individual=min_mutation_individual,
                                     mutation_rate=mutation_rate,
                                     verbose=verbose)
        prog_bar.update(1)
    
    prog_bar.close()
    
    if return_counter:
        str_snapshot = [''.join(x) for x in population_list]
        strat_df = pd.DataFrame.from_dict(Counter(str_snapshot),
                                          orient='index').reset_index()
        strat_df.rename(columns={'index': 'Strategy',
                                 0: 'Population'},
                        inplace=True)
        strat_df['Ratio'] = round(strat_df['Population'] / sum(strat_df['Population']), 2)
        strat_df['C_%'] = strat_df.apply(count_c, axis=1)
        strat_df.sort_values(by='Population',
                             ascending=False,
                             inplace=True)
        strat_df.reset_index(drop=True,
                             inplace=True)
        return strat_df
        
    else:
        return population_list


### Run some simulations, from simple ones.

In [26]:
genetic_selection(total_n=8,
                  past_n = 8,
                  population_size=10,
                  max_competitions_per_round=50,
                  evolution_cycles=5,
                  mutation_rate=0.1,
                  crossover_rate=.2,
                  verbose=2)

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-5, 10)"
1,Defect,"(10, -5)","(-1, -1)"



-----
Generation: 0


,Strategy,Freq,Avg_wt,New_freq,C_%
0,CCCCCDCC,1,1963,3,0.88
1,CDDCCCCC,1,1780,0,0.75
2,CCDCCDCC,1,1726,1,0.75
3,DCCDCDDC,1,1548,0,0.50
4,CCDCDDCC,1,1504,2,0.62
5,CDCDDDCC,1,1403,3,0.50
6,DCDCDDCC,1,1366,1,0.50
7,CCDCDDDC,1,1327,0,0.50
8,DDDCDCDD,1,1212,0,0.25
9,CDCCDDDD,1,1157,0,0.38



2 of 10 crossovers.
father: CCDCDDCC
mother: CCCCCDCC
 child: CCCCCDCC

father: CCCCCDCC
mother: CCDCDDCC
 child: CCDCDDCC


Mutation:
CCC(C->D)CDCC

-----
Generation: 1


,Strategy,Freq,Avg_wt,New_freq,C_%
0,CCCCCDCC,2,1906.5,3,0.88
1,CCCDCDCC,1,1764.0,1,0.75
2,CCDCCDCC,1,1725.0,2,0.75
3,CCDCDDCC,2,1512.0,0,0.62
4,DCDCDDCC,1,1408.0,1,0.50
5,CDCDDDCC,3,1362.0,3,0.50


...

,Strategy,Freq,Avg_wt,New_freq,C_%
0,CCCCCDCC,1,1917.00,0,0.88
1,CCCDCDCC,1,1745.00,1,0.75
2,CCDCCDCC,2,1744.50,3,0.75
3,CDCDCDCC,3,1577.33,5,0.62
4,CCCDDDCC,1,1548.00,0,0.62
5,DCDCDDCC,2,1479.00,1,0.50



2 of 10 crossovers.
father: CCCDCDCC
mother: CCCDCDCC
 child: CCCDCDCC

father: CCDCCDCC
mother: CCCDCDCC
 child: CCDCCDCC


Mutation:
CDCDC(D->C)CC
D(C->D)DCDDCC
CDCD(C->D)CCC
(C->D)DCDDCCC

-----
Generation: 4


,Strategy,Freq,Avg_wt,New_freq,C_%
0,CCDCCDCC,3,1850.0,3,0.75
1,CCCDCDCC,1,1804.0,0,0.75
2,DDCDDCCC,5,1374.2,7,0.50
3,DDDCDDCC,1,1165.0,0,0.38



2 of 10 crossovers.
father: CCDCCDCC
mother: DDCDDCCC
 child: CCCDDCCC

father: DDCDDCCC
mother: CCDCCDCC
 child: DDCCCDCC


Mutation:
(D->C)DCDDCCC



,Strategy,Population,Ratio,C_%
0,CDCDDCCC,6,0.6,0.62
1,CCDCCDCC,2,0.2,0.75
2,CCCDDCCC,1,0.1,0.75
3,DDCCCDCC,1,0.1,0.62


In [15]:
genetic_selection(total_n=8,
                  past_n=8,
                  population_size=12,
                  max_competitions_per_round=50,
                  evolution_cycles=20,
                  mutation_rate=0.03,
                  crossover_rate=0.5,
                  verbose=1)

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-5, 10)"
1,Defect,"(10, -5)","(-1, -1)"



-----
Generation: 0


,Strategy,Freq,Avg_wt,New_freq,C_%
0,DCCDCCDC,1,1737,2,0.62
1,CDCCCCCD,1,1654,1,0.75
2,CCCCDDCD,1,1556,0,0.62
3,CCDCDCCD,1,1437,1,0.62
4,CDCCCCDD,1,1420,0,0.62
5,CDDCCDDC,1,1344,2,0.50
6,DDCCDCCD,1,1221,0,0.50
7,DCDDDCCD,1,1169,0,0.38
8,DDCCDDCD,1,1114,2,0.38
9,DDDCCCDD,1,1070,1,0.38



Mutation:
DCCD(C->D)CDC

-----
Generation: 1


,Strategy,Freq,Avg_wt,New_freq,C_%
0,CDCCCCCD,1,1699.00,1,0.75
1,DCCDDCDC,1,1497.00,2,0.50
2,CCDCDCCD,1,1495.00,1,0.62
3,CDDCCDDC,3,1142.67,3,0.50
4,DCCDCDDD,1,1139.00,1,0.38
5,DDCCDDCD,2,1126.50,1,0.38
6,DDDDDCDC,1,891.00,1,0.25
7,CDDDCDDD,2,707.50,2,0.25


...

,Strategy,Freq,Avg_wt,New_freq,C_%
0,CCCCDCCC,4,1966.50,4,0.88
1,CDCCDCCC,1,1897.00,1,0.75
2,CCDCDCCC,7,1735.57,7,0.75



Mutation:
CCDCD(C->D)CC

-----
Generation: 19


,Strategy,Freq,Avg_wt,New_freq,C_%
0,CCCCDCCC,5,1961.4,6,0.88
1,CDCCDCCC,1,1888.0,3,0.75
2,CCDCDCCC,4,1750.5,2,0.75
3,CCDCDDCC,2,1659.0,1,0.62



Mutation:
CDCCDCC(C->D)



,Strategy,Population,Ratio,C_%
0,CCCCDCCC,6,0.50,0.88
1,CDCCDCCC,2,0.17,0.75
2,CCDCDDCC,2,0.17,0.62
3,CCDCDCCC,1,0.08,0.75
4,CDCCDCCD,1,0.08,0.62


### A cooperative strategy predominated that simulation. Too small of a population size, and evolutionary path can play out in other directions, as the example below shows.

In [19]:
genetic_selection(total_n=8,
                  past_n=8,
                  population_size=12,
                  max_competitions_per_round=50,
                  evolution_cycles=20,
                  mutation_rate=0.03,
                  crossover_rate=0.5,
                  verbose=1)

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-5, 10)"
1,Defect,"(10, -5)","(-1, -1)"



-----
Generation: 0


,Strategy,Freq,Avg_wt,New_freq,C_%
0,CCDCCDCD,1,1614,1,0.62
1,DCCCDCDC,1,1581,3,0.62
2,CDCDCCDC,1,1580,1,0.62
3,DDCCDCDC,1,1353,2,0.50
4,CCDCDDCD,1,1333,0,0.50
5,DCCDCDCD,1,1327,1,0.50
6,DDCCCDCD,1,1211,0,0.50
7,DDDDCCDC,1,1119,2,0.38
8,CDDCDCDD,1,1093,1,0.38
9,DDDDDDCC,1,911,1,0.25



Mutation:
DDDDDCD(C->D)

-----
Generation: 1


,Strategy,Freq,Avg_wt,New_freq,C_%
0,CCDCCDCD,1,1654.0,2,0.62
1,DCCCDCDC,1,1440.0,0,0.62
2,CDCDDDCC,1,1306.0,1,0.50
3,DCCDCDCD,1,1202.0,0,0.50
4,DDCDCCDC,1,1143.0,3,0.50
5,DDCCDCDC,1,1139.0,1,0.50
6,CDDCDCDD,1,1035.0,2,0.38
7,DCDDDDCC,2,1025.0,2,0.38
8,DDDDCCDC,2,966.5,1,0.38
9,DDDDDCDD,1,586.0,0,0.12


...

,Strategy,Freq,Avg_wt,New_freq,C_%
0,CCDCCDCC,1,1532.00,2,0.75
1,CCDCDDCC,7,1201.71,7,0.62
2,DCDCDDCC,4,1064.25,3,0.50



Mutation:
CCDCDD(C->D)C

-----
Generation: 19


,Strategy,Freq,Avg_wt,New_freq,C_%
0,CCDCCDCC,1,1545.00,2,0.75
1,CCDCDDCC,7,1206.86,8,0.62
2,CCDCDDDC,1,1096.00,0,0.50
3,DCDCDDCC,3,1066.00,2,0.50



Mutation:
DCDCCDC(C->D)



,Strategy,Population,Ratio,C_%
0,CCDCDDCC,9,0.75,0.62
1,DCDCDDCC,1,0.08,0.50
2,CCDCCDCC,1,0.08,0.75
3,DCDCCDCD,1,0.08,0.50


In [839]:
evolutionary_snapshot = genetic_selection(total_n=20,
                                          past_n=20,
                                          population_size=100,
                                          max_competitions_per_round=100,
                                          evolution_cycles=100,
                                          min_mutation_individual=2,
                                          mutation_rate=0.03,
                                          crossover_rate=0.5,
                                          verbose=0)

evolutionary_snapshot

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-5, 10)"
1,Defect,"(10, -5)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,CCDCCCCCDCCCCCCCCCCC,13,0.13,0.90
1,CCDCCCCCDDCCCCCCCDCC,6,0.06,0.80
2,CCCCCCCCDCCCCCCCCCCC,5,0.05,0.95
3,CCCCCCCCDCDCCCCDCCCC,5,0.05,0.85
4,CCDCCCCCDCDCCCCDCCCC,4,0.04,0.80
5,CCDCCCCCDCDCCCCCCDCC,4,0.04,0.80
6,CCDCCCCCDCCCCCCDCCCC,4,0.04,0.85
7,CCCCCCCCDCCCCCCDCCCC,4,0.04,0.90
8,CCDCCCCCDCCCCCCCCDCC,4,0.04,0.85
9,CCDDDDCCCCDCDCCDCCCC,3,0.03,0.65


In [840]:
evolutionary_snapshot_1 = genetic_selection(total_n=50,
                                          past_n=50,
                                          population_size=100,
                                          max_competitions_per_round=50,
                                          evolution_cycles=100,
                                          min_mutation_individual=2,
                                          mutation_rate=0.03,
                                          crossover_rate=0.5,
                                          verbose=0,
                                          return_counter=True)

evolutionary_snapshot_1

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-5, 10)"
1,Defect,"(10, -5)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,CCCCCDCCCCCDCDDCCCDDDCCCCDCCCCCCDDCDCDDCCDCCCDCCDC,6,0.06,0.68
1,CCCCDDCCDCCCCDCCCDDCDCCCCDCCCCCCCCCDCDDCCCCCCDCCDC,5,0.05,0.74
2,CCCCCCDCCCCDDDCCCDDDDCCCDCCCCCCCCCCDCDDCCDCCCDCCDC,3,0.03,0.70
3,CCCCCCDCCCCDDDCCCDDDDCCCDCCCCCDDCCCCCDCDCCCDCCCDDC,3,0.03,0.68
4,CCCCCCDCCCCDCDCCCDDDDCCCCDCCCCCCCCCCCDCDCCCDCCCDCC,3,0.03,0.76
5,CCCCCDCCCCCCDDDCDCCDDCCCCCCCCCCCCCCDCDCDCCCDCCCDCC,2,0.02,0.76
6,CCCCCCDCCCCDCDDCDCCDCDCCCCCCCCCCCCDCCDCCCDCDCDCCDC,2,0.02,0.74
7,CCCCCDCCCCCCDDDCDCDDDCCCCDCCCCCCCCCDCDDCCCCCCDCCDC,2,0.02,0.72
8,CCCCCCDCCCCCCDDCDCCCDCCCCDCCCCCCCCCDCDDCCCCCCDCCDC,2,0.02,0.78
9,CCCCCDCCCCCDCDDCCDDDDCCCCCCCCCCCCCCDCDCDCCCDCCCDCC,2,0.02,0.74


In [841]:
evolutionary_snapshot_2 = genetic_selection(total_n=50,
                                            past_n=50,
                                            population_size=100,
                                            max_competitions_per_round=50,
                                            evolution_cycles=100,
                                            min_mutation_individual=2,
                                            mutation_rate=0.03,
                                            crossover_rate=0.5,
                                            verbose=0,
                                            return_counter=True)

evolutionary_snapshot_2

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-5, 10)"
1,Defect,"(10, -5)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,CCCDCCCCDCCCCCCCCCDDCCDCCCCCDCCDCCCCCCCDCCDCCCCCCC,4,0.04,0.82
1,DCCCCCCCDCCCCCCDCDCCCCDCDCCCDCCDCDCCCCCCCCDCCCDCCC,3,0.03,0.78
2,DCCCCCCCDCCCCCCDCDCCCCDCCCCCCCCDCDCCCCCCCCDCCCCCCC,3,0.03,0.84
3,CCCDCCCCDCCCCCCCCCDDCCDCCCCCDCCDCCCDCCCDCCDCCCCCCC,2,0.02,0.80
4,CCCDCCCDDCCCCDCCCCCCCCCCCCCCDCDDCDCCCCCCCCDCCCCCDC,2,0.02,0.80
5,DCCCCCCCDCCCCCCDCDCCCCDCCCCCCCCDCDCCCCCCCCCCCCCCDC,2,0.02,0.84
6,DCCDCCCDCDDCCDCCCCCCCCCCCDCCCCCDCCCCCCCCCCDCCCCCDC,2,0.02,0.80
7,CCCDCCCDDDDCCDCCDCCCCCCCCDCCDCCDCDCDCDDCCCDCCCCCDC,2,0.02,0.68
8,CCCDCCCDDCCCCCCCDCCCCCCCCCCCDCDDCDCCDCCCCCDCCCCCCC,2,0.02,0.80
9,DCCDCCCDDCCCCDCCCCCCCCDCCDCCCCCDCDCCCCCCCCCCCCCCDC,2,0.02,0.80


In [842]:
evolutionary_snapshot_3 = genetic_selection(total_n=50,
                                            past_n=50,
                                            population_size=100,
                                            max_competitions_per_round=50,
                                            evolution_cycles=100,
                                            min_mutation_individual=2,
                                            mutation_rate=0.03,
                                            crossover_rate=None,
                                            verbose=0,
                                            return_counter=True)

evolutionary_snapshot_3

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-5, 10)"
1,Defect,"(10, -5)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,CCDCCCDDCDCCDDCCCCCDDCCCDCDDDCCCCCDDCCDCDDDDCCCCDC,51,0.51,0.60
1,DDCCCDDCDDDCCDDCDCDDDCCCDDCDDDDDDDDCCCDCCCDDDCCCCC,29,0.29,0.46
2,DDDCDCCDDDDCCCDCCCDCDDDCCDDDDDCDDDCCCDDCCCDDCDDDDC,20,0.20,0.42


In [843]:
evolutionary_snapshot_4 = genetic_selection(total_n=50,
                                            past_n=50,
                                            population_size=100,
                                            max_competitions_per_round=50,
                                            evolution_cycles=100,
                                            min_mutation_individual=2,
                                            mutation_rate=0.03,
                                            crossover_rate=None,
                                            verbose=0,
                                            return_counter=True)

evolutionary_snapshot_4

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-5, 10)"
1,Defect,"(10, -5)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,DCCDDDCCCCDDCDCCCDDCCDCDDCDCCCCCDCCDDCDDCDDCDCCDCD,40,0.40,0.54
1,CCCCDCCDDCDCCCDCCDDDCCDDCCCCDDDDDCCCCCCDCDCCDDCDDD,35,0.35,0.56
2,DDCCDDCDCCCCDDDDCCDCCDCCCCDDCDDCCDCCDDDDCCDCDDCCCD,25,0.25,0.52


In [844]:
evolutionary_snapshot_5 = genetic_selection(total_n=50,
                                            past_n=50,
                                            population_size=100,
                                            max_competitions_per_round=50,
                                            evolution_cycles=100,
                                            min_mutation_individual=2,
                                            mutation_rate=0.5,
                                            crossover_rate=1,
                                            verbose=0,
                                            return_counter=True)

evolutionary_snapshot_5

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-5, 10)"
1,Defect,"(10, -5)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,CCDCDDCCCDDDDCCCCDCDCCDCDDDDDCCCCCDCDDCCCCDDCCDCCC,2,0.02,0.58
1,CCCCCDCDCDCCCDCCCDDDDCCCDCDCDCCCCCDCCCDCCDCDDCDCCC,1,0.01,0.66
2,CDCCDDDCCDDCCCCCCDDDCCDCDCCCDDDCCCCCCCDCCCCDCDCDCC,1,0.01,0.64
3,CCCDCDCDCCCCCCDCCDDDCDCCDCCCDCCCCCCCCCDCDDCDCDCCCC,1,0.01,0.70
4,CCCCCDCCCCDDDCDCDDDCDCCCCCDCDCCCCCDCCCDCCCCDCCDCCC,1,0.01,0.70
5,CCDCCDCCCDDDDCDCCCDDDCDCDCDDDDDCCCDCCCDCCCCDCCDDCD,1,0.01,0.54
6,CCCCDDCDCCCCCDDCCDDDCCDCCCCDDDDCCDCCCCDDCCCDCCCCCC,1,0.01,0.66
7,CCCCDCDDCCDDCCDCCCDDCDCCDCCCCCCCCCCCCCDCDDCCCDDCCC,1,0.01,0.70
8,CCCCDDDCCDDCDCCCCDDDCDDCDCCCCDDCCCCCCCDCCCCDCDDCCC,1,0.01,0.64
9,CCCCDDCCCCCCDDCDCDDDDDCCCCDCDCCDCDDDCDDCCCCDDCCCCC,1,0.01,0.60


In [845]:
evolutionary_snapshot_6 = genetic_selection(total_n=20,
                                            past_n=20,
                                            population_size=100,
                                            max_competitions_per_round=200,
                                            evolution_cycles=100,
                                            min_mutation_individual=0, # accept zero mutation per ecolutionary cycle
                                            mutation_rate=0.01, # low mutation
                                            crossover_rate=1, # 100% sexual reproduction
                                            verbose=0,
                                            return_counter=True)

evolutionary_snapshot_6

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-5, 10)"
1,Defect,"(10, -5)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,CCCCCCCCCCCCCCDCDCCC,9,0.09,0.90
1,CCCCCDCCCCCCCCDCDDCC,6,0.06,0.80
2,CCCDCDCCCCCCCCDCDDCC,6,0.06,0.75
3,CCCCCCCCCDCCCCDCCCCC,4,0.04,0.90
4,CDCCCCCCCCCCCCDCDCCC,4,0.04,0.85
5,CCCDCCCCCCCCCCDCDCCC,4,0.04,0.85
6,CDCCCCCCCCCDCCDCDDCC,4,0.04,0.75
7,CCCCCDCCCCCCCCDCDCCC,3,0.03,0.85
8,CCCCCCCCCCCCCCDCDDCC,3,0.03,0.85
9,CCCCCCCCCDDCCCDCDDCC,3,0.03,0.75


In [846]:
evolutionary_snapshot_7 = genetic_selection(total_n=20,
                                            past_n=20,
                                            population_size=100,
                                            max_competitions_per_round=200,
                                            evolution_cycles=100,
                                            min_mutation_individual=0, # accept zero mutation per ecolutionary cycle
                                            mutation_rate=0.01, # low mutation
                                            crossover_rate=1, # 100% sexual reproduction
                                            verbose=0,
                                            return_counter=True)

evolutionary_snapshot_7

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-5, 10)"
1,Defect,"(10, -5)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,CCCCCCDCCCCCCCDDCCCC,15,0.15,0.85
1,CCCCCCCCCCCCCCDDCCCC,10,0.10,0.90
2,CCCCCCDCCCCCCCDCCCCC,9,0.09,0.90
3,CCCCCCCCCCCCCCDDCCDC,7,0.07,0.85
4,CCCCCCCCCCCCCCDCCCCC,6,0.06,0.95
5,CCCCCCDCCCCCCCDDCCDC,6,0.06,0.80
6,CCCCCCCCCCCCCCDCCCDC,5,0.05,0.90
7,CCCCCCDCCCCCDCDCCCDC,5,0.05,0.80
8,CCDCCCCCCCCCCCDDCCDC,4,0.04,0.80
9,CCCCCCDCCCCCDCDCCCCC,4,0.04,0.85


In [847]:
evolutionary_snapshot_8 = genetic_selection(total_n=20,
                                            past_n=20,
                                            population_size=100,
                                            max_competitions_per_round=10,
                                            evolution_cycles=500,
                                            min_mutation_individual=1, # accept zero mutation per ecolutionary cycle
                                            mutation_rate=0.01, # low mutation
                                            crossover_rate=1, # 100% sexual reproduction
                                            verbose=0,
                                            return_counter=True)

evolutionary_snapshot_8

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-5, 10)"
1,Defect,"(10, -5)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,CCCCCCCCCCCCCCCCCCCC,58,0.58,1.00
1,CCCCDCCCCCCCCCCCCCCC,19,0.19,0.95
2,CCCCCCCCDCCCCCCCCCCC,8,0.08,0.95
3,CCCCCCCCCCCCCCCCCCDC,7,0.07,0.95
4,CCCCDCCCCCCCCCCCCCDC,3,0.03,0.90
5,CDCCCCCCCCCCCCCCCCCC,2,0.02,0.95
6,CCCCCCCCDCCCCCCCCCDC,1,0.01,0.90
7,CDCCCCCCCCCCCCCCCCDC,1,0.01,0.90
8,CCCCDCDCCCCCCCCCCCCC,1,0.01,0.90


### While a strictly cooperative strategy may not intuitively be competitive, the "fitness" is always contextual. In a culture where almost everyone cooperates, it pays to cooperate w/ others. An emergence of a cooperative culture.
### Defectors can subvert this culture, but two factors prohibit this:
### 1. There's no outsider in this simulation. All players are descended from cooperative parents. In this simulation, mutation is rare.
### 2. A critical mass of defectors is more likely to change the existing cooperative culture.

### Some more simulations:

In [848]:
evolutionary_snapshot_9 = genetic_selection(total_n=20,
                                            past_n=20,
                                            population_size=100,
                                            max_competitions_per_round=50,
                                            evolution_cycles=1000, # More cycles
                                            min_mutation_individual=5,
                                            mutation_rate=0.01,
                                            crossover_rate=1, # 100% sexual reproduction
                                            verbose=0,
                                            return_counter=True)

evolutionary_snapshot_9

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-5, 10)"
1,Defect,"(10, -5)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,CDCCCDCCDCCCCCCCCCCC,8,0.08,0.85
1,CCCCCCCCDCCCCCCCCCCC,8,0.08,0.95
2,CCCCCDCCDCCCCCCCCCCC,8,0.08,0.90
3,CCCCCDCCDCCCCCCCCCCD,6,0.06,0.85
4,CDCCCCCCDCCCCCCCCCCC,4,0.04,0.90
5,CCCCCDCCDCCCCCCCCCDC,4,0.04,0.85
6,CCCCCCCCDCCCCCCCCCCD,3,0.03,0.90
7,CCCDDDCCDDCCCCCCCCCC,3,0.03,0.75
8,CCCCCCCCDCCCCCCCCCDC,3,0.03,0.90
9,CCDCCCCCDCCCCCCCCCCC,3,0.03,0.90


### Try a few different payoff matrixes

In [849]:
low_penalty_matrix = {'CC': (3, 3), 'CD': (0, 10), 'DC': (10, 0), 'DD': (-1, -1)} # less penalty for being cheated

evolutionary_snapshot_10 = genetic_selection(matrix=low_penalty_matrix,
                                             total_n=20,
                                            past_n=20,
                                            population_size=100,
                                            max_competitions_per_round=50,
                                            evolution_cycles=1000,
                                            min_mutation_individual=5,
                                            mutation_rate=0.01,
                                            crossover_rate=1, # 100% sexual reproduction
                                            verbose=0,
                                            return_counter=True)

evolutionary_snapshot_10

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(0, 10)"
1,Defect,"(10, 0)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,CCCCCCCCCCDCCCCDCCCC,4,0.04,0.90
1,DCCCCDCCCCCCCCCCCCCD,3,0.03,0.85
2,CCCCCCCCCCCCCCCCCCCD,3,0.03,0.95
3,CCCCCDCCCCCCCCCCCCCC,3,0.03,0.95
4,CCCCCDCCCCCCCCCCCCCD,2,0.02,0.90
5,CCDCCDCCCCCCDCCCCCCD,2,0.02,0.80
6,DCCCCDCCCCCCCCCCCCCC,2,0.02,0.90
7,CCCCCDCCCDDCCCDDCCCD,2,0.02,0.70
8,CCCCCCCCDCDCCCCCCCCC,2,0.02,0.90
9,CCCCCCCCDCCCDCCCDCCC,2,0.02,0.85


In [850]:
severe_penalty_matrix = {'CC': (3, 3), 'CD': (-50, 5), 'DC': (5, -50), 'DD': (-1, -1)} # less penalty for being cheated


genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=1000,
                  min_mutation_individual=5,
                  mutation_rate=0.01,
                  crossover_rate=1, # 100% sexual reproduction
                  verbose=0,
                  return_counter=True)

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-50, 5)"
1,Defect,"(5, -50)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,DCCCCDCCCDDDCDDDCCCD,95,0.95,0.55
1,DCDCCDCCCDDDCDDDCCCD,2,0.02,0.50
2,DCCCDDCCCDDDCDDDCCCD,1,0.01,0.50
3,DCCCCDCCCDDDCDDCCCCD,1,0.01,0.60
4,DCCCCDCCCDDDDDDDCCCD,1,0.01,0.50


In [851]:
reverse_penalty_matrix = {'CC': (3, 3), 'CD': (10, 0), 'DC': (0, 10), 'DD': (-1, -1)} # less penalty for being cheated


genetic_selection(matrix=reverse_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=5,
                  mutation_rate=0.01,
                  crossover_rate=1, # 100% sexual reproduction
                  verbose=0,
                  return_counter=True)

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(10, 0)"
1,Defect,"(0, 10)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,CDCCCCCDCDCDCCCCCDDC,3,0.03,0.70
1,CCCCCCCDCCDDCCDCCDCD,2,0.02,0.70
2,CCDDCDCDCCCDCDCCCDDC,2,0.02,0.60
3,CCCDCDCDCCCDCCDCCDDD,2,0.02,0.60
4,CCCDDCCDCCCCCCCCCCDC,2,0.02,0.80
5,CCCCCCCCCCCDCDDCCCDD,2,0.02,0.75
6,CCCDCCDDCDCDCCDCCDDC,2,0.02,0.60
7,CCDCDCDDCDCCCDCCCDDD,2,0.02,0.55
8,CCCCCCCDCDDCCCDCCDDC,2,0.02,0.70
9,CCCCCCDDCCCDCCCCCDDC,2,0.02,0.75


In [853]:
cooperative_matrix = {'CC': (30, 30), 'CD': (10, 0), 'DC': (0, 10), 'DD': (-1, -1)} # less penalty for being cheated

# Sexual selection, low mutation

genetic_selection(matrix=cooperative_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=1,
                  verbose=0,
                  return_counter=True)

,Action,Cooperate,Defect
0,Cooperate,"(30, 30)","(10, 0)"
1,Defect,"(0, 10)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,CCCCCCCCDCCCCCCCDCCD,58,0.58,0.85
1,CCCCDCCCDCCCCCCCDCCD,16,0.16,0.80
2,CCCCCCCCDCCCCCCCCCCD,8,0.08,0.90
3,CCCCCCCDDCCCCCCCDCCD,8,0.08,0.80
4,CCCCCCCDDCCCCCCCCCCD,2,0.02,0.85
5,CCCCDCCCDCCCCCCCCCCD,2,0.02,0.85
6,CDCCCCCCDCCCCCCCDCCD,2,0.02,0.80
7,CCCCCCCCDCCCCCCCDCCC,1,0.01,0.90
8,CCCCDCCCDCCCCDCCDCCD,1,0.01,0.75
9,CCCCDCCDDCCCCCCCDCCD,1,0.01,0.75


In [854]:
# Asexual reproduction, low mutation rate

genetic_selection(matrix=cooperative_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=0,
                  verbose=0,
                  return_counter=True)

,Action,Cooperate,Defect
0,Cooperate,"(30, 30)","(10, 0)"
1,Defect,"(0, 10)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,DCCCCCCCCDCDCDCDCCCD,100,1.0,0.7


In [855]:
# Asexual reproduction, low mutation rate, longer evolutionary cycles

genetic_selection(matrix=cooperative_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=500,
                  evolution_cycles=200, # +
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=0,
                  verbose=0,
                  return_counter=True)

,Action,Cooperate,Defect
0,Cooperate,"(30, 30)","(10, 0)"
1,Defect,"(0, 10)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,DDDCCDCCCCCCDDDCDDCC,100,1.0,0.55


### It seems (from a small sample size! Despite running cells above a few times) that crossover enhances the building of strategies w/ useful foundational blocks. Or it allows for more diversity, which quickly allows cooperative strategies to emerge.

In [857]:
# Asexual reproduction, high mutation rate

genetic_selection(matrix=cooperative_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.2,
                  crossover_rate=0,
                  verbose=0,
                  return_counter=True)

,Action,Cooperate,Defect
0,Cooperate,"(30, 30)","(10, 0)"
1,Defect,"(0, 10)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,DDDDCDDDCDCCDCCDCCCC,100,1.0,0.5


In [859]:
genetic_selection(matrix=cooperative_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.2,
                  crossover_rate=0,
                  verbose=0,
                  return_counter=True)

,Action,Cooperate,Defect
0,Cooperate,"(30, 30)","(10, 0)"
1,Defect,"(0, 10)","(-1, -1)"


,Strategy,Population,Ratio,C_%
0,DDCDCDCDDDDCCDDCDCCC,100,1.0,0.45


In [559]:
genetic_selection(matrix=cooperative_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=5, # +
                  mutation_rate=0.01,
                  crossover_rate=0,
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
0,DCDCCCCCCCDDDCCDCDDD,100,1.0,11


In [757]:
genetic_selection(matrix=cooperative_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=1, # +
                  verbose=0,
                  return_counter=True)

,Action,Cooperate,Defect
0,Cooperate,"(30, 30)","(10, 0)"
1,Defect,"(0, 10)","(-1, -1)"


,Strategy,Population,Ratio,C_count
0,CCDCDCCCCCCCCCCCCCCC,45,0.45,18
1,CCDCDCCCCCCCCCCCCCCD,38,0.38,17
2,CDDCDCCCCCCCCCCCCCCD,5,0.05,16
3,CDDCCCCCCCCCCCCCCCCD,3,0.03,17
4,CCDCDCCCCCCCCCCDCCCC,2,0.02,17
5,CDDCDCCCCCCCCCCCCCCC,2,0.02,17
6,CCDCDCCCDCCCCCCCCCCC,1,0.01,17
7,CCDDDCCCCCCCCCCCCCCC,1,0.01,17
8,CCDCDCCCDCCCCCCCCCCD,1,0.01,16
9,CCDCCCCCCCCCCCCCCCCC,1,0.01,19


In [758]:
genetic_selection(matrix=cooperative_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=0.5, # +
                  verbose=0,
                  return_counter=True)

,Action,Cooperate,Defect
0,Cooperate,"(30, 30)","(10, 0)"
1,Defect,"(0, 10)","(-1, -1)"


,Strategy,Population,Ratio,C_count
0,CCCCCCDCCCCCCDCCDCCC,37,0.37,17
1,CCCCCCDCCCCCCCCCDCCC,27,0.27,18
2,CCCCDCDCCCCCCDCCDCCC,8,0.08,16
3,CCCCCCDCCCCCDDCCDCCC,6,0.06,16
4,CCCCCDDCCCCCCCCCDCCC,4,0.04,17
5,CCCCCCDCCCCCCDCCDCCD,3,0.03,16
6,CCCCCCDCCCCCCDCDDCCD,2,0.02,15
7,CCCCCCDCCCCCCDCCDDCC,2,0.02,16
8,CCCCCDDCCCCCCDCCDCCC,2,0.02,16
9,CCCCCCDCCCCCCCCCDCCD,2,0.02,17


In [564]:
# Baseline, severe_penalty_matrix
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=0,
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
0,DDCCCDDCDCCDCCDDDCCD,100,1.0,10


In [565]:
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=1, # +
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
0,CCDCDDDDCDCDDDDDDDDD,100,1.0,5


In [566]:
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=200, # +
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=0,
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
0,DDDDCDCDDDDCDCDDCCDD,200,1.0,6


In [567]:
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=100, # +
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=0,
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
0,CDDCDCCDCDDCCCDDCDCD,100,1.0,10


In [568]:
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=500, # +
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=0,
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
0,CDCDDDDCCCCCDDDCDDDC,100,1.0,9


In [569]:
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.1, # +
                  crossover_rate=0,
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
0,DCCCDCCCDCCCDCCDCDDC,100,1.0,13


In [570]:
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.1,
                  crossover_rate=0,
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
0,CCCDDDCCCDDDDDDDDDDC,100,1.0,7


In [571]:
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.1,
                  crossover_rate=0,
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
0,CCCDCCDDDDDCDCCCCCCD,100,1.0,12


### Looks like a high mutation_rate will yield highly variable nash equilibria.

In [572]:
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=1,
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
1,DDCDDCCDDCCDDDCDDDDD,69,0.69,6
0,DDCDDCCDCCCDDDCDDDDD,17,0.17,7
2,DDDDDCCDCCCDDDCDDDDD,12,0.12,6
3,DDCDDCCDDCCDDDCDDDCD,1,0.01,7
4,DDCDDCCDDCCDCDCDDDDD,1,0.01,7


In [591]:
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=.5,
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
0,CDDDDDDDDDCCDDDDDDDD,98,0.98,3
1,CDDDDDDDDDCCDDDDDDCD,1,0.01,4
2,CDDDDDDDDDCCDDDDDDDC,1,0.01,4


### Ran the simulation above several times, it seemed that a moderate crossover_rate will be even better than extreme values (0 or 1).

In [598]:
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=0,
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
0,DCDCCCCDCCCDCCDCCDDD,100,1.0,12


In [604]:
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=1,
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
0,DDDDCDCCDDDDCDCDDDDC,97,0.97,6
1,DDDDCDCCDDDDCDCDDDCC,1,0.01,7
2,DDDDCDCCDDDDCDDDDDDC,1,0.01,5
3,DDDDCDCCDDDDCDCCDDDC,1,0.01,7


In [636]:
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=1000,
                  max_competitions_per_round=50,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=0.5,
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
0,DDCDDCDDDCDCCCDCCCCC,449,0.45,11
2,DDDDCCDDDDDCCDCDCCCD,413,0.41,8
1,DDDCCCCDDDCCCDCCDCCD,51,0.05,11
5,DDDCCCCDDDDCCDCCDCCD,20,0.02,10
4,DDDDCCDDDDDCCDCCDCCD,19,0.02,8
3,DDDCCCCDDDDCCDCDCCCD,17,0.02,10
6,DDDCCCDDDDDCCDCDCCCD,15,0.02,9
8,DDDDCCCDDDDCCDCCDCCD,7,0.01,9
7,DDCDCCDDDDDCCDCDCCCD,3,0.00,9
9,DDDDCCDDDDDCCCCDCCCD,2,0.00,9


In [654]:
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=100,
                  max_competitions_per_round=5,
                  evolution_cycles=100,
                  min_mutation_individual=0,
                  mutation_rate=0.01,
                  crossover_rate=0.5,
                  verbose=0,
                  return_counter=True)

,Strategy,Population,Ratio,C_count
0,CCDCDCCDDCDCDDCDDCDD,90,0.9,9
1,CCDCDCCDDDDCDDCDDCDD,10,0.1,8


### Ran the cell above a few times, it seemed that increasing max_competitios_per_round past some fairly low threshold has limited benefits. It seemed to suggest that every individual having lots of experience is unnecessary. Nature has sown so many seeds, the suitable ones will persist nonetheless.

In [723]:
genetic_selection(matrix=severe_penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=50,
                  max_competitions_per_round=20,
                  evolution_cycles=500,
                  min_mutation_individual=0,
                  mutation_rate=0.05,
                  crossover_rate=0.9,
                  verbose=0,
                  return_counter=True)

,Action,Cooperate,Defect
0,Cooperate,"(3, 3)","(-50, 5)"
1,Defect,"(5, -50)","(-1, -1)"


,Strategy,Population,Ratio,C_count
0,DDDCDDDCCDDDDCDCDDDD,48,0.96,5
1,DCDCDDDCCDDDDCDCDDDD,1,0.02,6
2,DDDCDDDCDDDDDCDCDDDD,1,0.02,4


In [759]:
penalty_matrix = {'CC': (-50, -50), 'CD': (-50, 50), 'DC': (50, -50), 'DD': (50, 50)}
genetic_selection(matrix=penalty_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=50,
                  max_competitions_per_round=20,
                  evolution_cycles=500,
                  min_mutation_individual=0,
                  mutation_rate=0.05,
                  crossover_rate=0.9,
                  verbose=0,
                  return_counter=True)

,Action,Cooperate,Defect
0,Cooperate,"(-50, -50)","(-50, 50)"
1,Defect,"(50, -50)","(50, 50)"


,Strategy,Population,Ratio,C_count
0,DDDDDDDDDDDDDDDDDDDD,11,0.22,0
1,DDDDDDCDDDDDDDDDDDDD,5,0.10,1
2,DDDDDDDDDDDDDDDDDDCD,4,0.08,1
3,DDDDDDDDDCDDDDDDDCDD,4,0.08,2
4,DCDDDDDDDDDDDDDDDDDD,4,0.08,1
5,DDDDDDCDDCDDDDDDDDDD,3,0.06,2
6,DDDDDDDDDDDDDDDDDCDD,3,0.06,1
7,DDDDDDCDDCDDDDDDDCCD,2,0.04,4
8,DDDDDDDDDDDDDDDDCCDD,2,0.04,2
9,DCDDDDCDDDDDDDDDDDDD,2,0.04,2


In [761]:
genetic_selection(matrix=cooperative_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=50,
                  max_competitions_per_round=20,
                  evolution_cycles=500,
                  min_mutation_individual=0,
                  mutation_rate=0.05,
                  crossover_rate=0.9,
                  verbose=0,
                  return_counter=True)

,Action,Cooperate,Defect
0,Cooperate,"(30, 30)","(10, 0)"
1,Defect,"(0, 10)","(-1, -1)"


,Strategy,Population,Ratio,C_count
0,CCCCCCCCCDCCCCCDCCCC,25,0.50,18
1,CCCCCCCCCDCCCCCDCDCC,5,0.10,17
2,CCCCCCCCCDCCCCCDDDCC,3,0.06,16
3,CCCCCCDCCDCCCCCDCCCC,3,0.06,17
4,DDCCCCCCCDCCCCCDCCCC,2,0.04,16
5,CCCCCCDCCDCCCCCDCDCC,2,0.04,16
6,CCCCCCCCCDCCDCCDCCCC,1,0.02,17
7,CCCCCCCCCDCCCCCDCDCD,1,0.02,16
8,DDCCCCCCCDCCCCCDCCDC,1,0.02,15
9,CCCCCCCCCDCDCCCDCDCC,1,0.02,16


In [764]:
one_sided_matrix = {'CC': (10, 0), 'CD': (-10, 10), 'DC': (0, -10), 'DD': (0, 0)}
genetic_selection(matrix=one_sided_matrix,
                  total_n=20,
                  past_n=20,
                  population_size=50,
                  max_competitions_per_round=20,
                  evolution_cycles=500,
                  min_mutation_individual=0,
                  mutation_rate=0.05,
                  crossover_rate=0.9,
                  verbose=0,
                  return_counter=True)

,Action,Cooperate,Defect
0,Cooperate,"(10, 0)","(-10, 10)"
1,Defect,"(0, -10)","(0, 0)"


,Strategy,Population,Ratio,C_count
0,CCCCCCCCCCCCCCCCCCCC,38,0.76,20
1,CDCCCCCCCCCCCCCCCCCC,4,0.08,19
2,CCCCCCCCDCCCCCCCCCCC,2,0.04,19
3,CCCCCCCCCCCCCCCCCDCC,1,0.02,19
4,CCCCCCDCCCCCCCCCCCCC,1,0.02,19
5,CCCCCCCCCCCCCCDCCCCC,1,0.02,19
6,CCCCCCCCCCCDCCCCCCCC,1,0.02,19
7,CCCCCCCCCDCCCCCCCCCC,1,0.02,19
8,CCDCCCCCCCCCCCCCCCCC,1,0.02,19


In [771]:
one_sided_matrix = {'CC': (10, 0), 'CD': (-10, 10), 'DC': (0, -10), 'DD': (0, 0)}
genetic_selection(matrix=one_sided_matrix,
                  total_n=20,
                  past_n=25,
                  population_size=10,
                  max_competitions_per_round=20,
                  evolution_cycles=20,
                  min_mutation_individual=0,
                  mutation_rate=0.05,
                  crossover_rate=0.9,
                  verbose=1,
                  return_counter=True)

,Action,Cooperate,Defect
0,Cooperate,"(10, 0)","(-10, 10)"
1,Defect,"(0, -10)","(0, 0)"



-----
Generation: 0
               Strategy  Freq  Avg_wt  New_freq  C_count
0  DDCCCDCDDCCCCCCCDCCC     1     840         2       14
1  DCDDCCCDCCDCDCDCCCCC     1     810         3       13
2  CCDCCDDCDCDCCDDDCDCC     1     500         2       11
3  CCDCDDDCDDDCCCDDCCDC     1     500         1       10
4  CDCCCCDCDDCCDCDDCDDD     1     400         1       10
5  DCDCCCDDCDDCDCCDDCCD     1     380         1       10
6  CCDDCDDDCCDCDCCDDCDC     1     330         0       10
7  DCDDCDDDCCCDDCCDCCDD     1      10         0        9
8  CCDDDDDCDDDCCCDDCDDD     1    -150         0        7
9  CDDCDCCDDDDDCCCCCDDC     1    -200         0       10

Mutations:

-----
Generation: 1
               Strategy  Freq  Avg_wt  New_freq  C_count
0  DDCCCDCDDCCCCCCCDCCC     2    1255         3       14
1  CCDCCDDCDCDCCDDCCCCC     1    1170         0       13
2  CCDCDDDCDDDCCCDCCCCC     1    1160         2       12
3  CDCCCCDCDDDCCCDDCCDC     1    1080         3       12
4  DCDDCCCDCCDCDCDCCCCC     1     

,Strategy,Population,Ratio,C_count
0,CDCCCDCDDCCCCCCDDCCC,8,0.8,14
1,CDCCCDCDDCCCCCCCDCCC,1,0.1,15
2,DDCCCDCDDCCCCCCDDCCC,1,0.1,13
